## 1 切割出目标位置

In [1]:
import cv2
import numpy as np
import os

def get_centered_crop_box(x_min, x_max, y_min, y_max, crop_width, crop_height, max_width, max_height):
    # 计算目标物体的中心点
    center_x = (x_min + x_max) // 2
    center_y = (y_min + y_max) // 2

    # 计算裁剪区域
    crop_x_min = max(center_x - crop_width // 2, 0)
    crop_x_max = crop_x_min + crop_width
    crop_y_min = max(center_y - crop_height // 2, 0)
    crop_y_max = crop_y_min + crop_height

    # 确保裁剪区域不超出原始图像边界
    if crop_x_max > max_width:
        crop_x_max = max_width
        crop_x_min = crop_x_max - crop_width
    if crop_y_max > max_height:
        crop_y_max = max_height
        crop_y_min = crop_y_max - crop_height

    return crop_x_min, crop_x_max, crop_y_min, crop_y_max

# 设置原始图像和mask图像的文件夹路径
original_images_path = 'image'
mask_images_path = 'mask'
# 设置裁剪后的图像保存的路径
cropped_images_path = 'cropped_image'
cropped_masks_path = 'cropped_mask'
if not os.path.exists(cropped_images_path):
    os.makedirs(cropped_images_path)
if not os.path.exists(cropped_masks_path):
    os.makedirs(cropped_masks_path)
# 设置裁剪后尺寸
crop_width = 512
crop_height = 512

# 遍历原始图像文件夹
for image_name in os.listdir(original_images_path):
    if image_name.endswith('.jpg'):
        original_image_path = os.path.join(original_images_path, image_name)
        mask_image_name = image_name.replace('.jpg', '.png')
        mask_image_path = os.path.join(mask_images_path, mask_image_name)

        original_image = cv2.imread(original_image_path)
        mask_image = cv2.imread(mask_image_path)

        # 转换mask图像为灰度以找到目标物体的位置
        mask_gray = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
        y_indices, x_indices = np.where(mask_gray > 0)
        x_min, x_max = x_indices.min(), x_indices.max()
        y_min, y_max = y_indices.min(), y_indices.max()

        max_height, max_width = original_image.shape[:2]
        crop_x_min, crop_x_max, crop_y_min, crop_y_max = get_centered_crop_box(x_min, x_max, y_min, y_max, crop_width, crop_height, max_width, max_height)

        cropped_image = original_image[crop_y_min:crop_y_max, crop_x_min:crop_x_max]
        cropped_mask = mask_image[crop_y_min:crop_y_max, crop_x_min:crop_x_max]
        cropped_image_path = os.path.join(cropped_images_path, image_name)
        cropped_mask_path = os.path.join(cropped_masks_path, mask_image_name)
        cv2.imwrite(cropped_image_path, cropped_image)
        cv2.imwrite(cropped_mask_path, cropped_mask)

## 2 分割图像

In [1]:
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
# 设置图像所在的文件夹目录
PATH_image = r'cropped_image'
PATH_mask = r'cropped_mask'
PATH = r'E:\workspace\Anaconda\Robot\InstanceSegmentation\dataset'

In [18]:
def extract_images_from_folder(PATH, folder, draw=False):
    """对图像进行批量分批处理（对文件夹中的所有图像进行分片）"""
    # 图像导入
    folder = os.path.join(PATH, folder)
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))

        if img is not None:
            # 如果导入成功，则创建该图片专属的文件夹
            subfolder = os.path.join(PATH, folder+'_split', filename.split('.')[0])
            if os.path.exists(subfolder) == False:
                os.mkdir(subfolder)
        
            # 图像分割
            img_copy = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).copy()
            img_h = img_copy.shape[0]
            img_w = img_copy.shape[1]

            patch_h = 200
            patch_w = 200

            for y in range(0, img_h, patch_h):
                for x in range(0, img_w, patch_w):
                    if patch_h > img_h or patch_w > img_w:
                        print('方片尺寸大于原始图像尺寸')
                        break
                    if img_h >= y + patch_h or img_w >= x + patch_w:
                        patch = img_copy[y:y+patch_h, x:x+patch_w]
                        if draw:
                            # 保存之后，在原始图像上对当前索引区域绘制白色边框
                            # 注意，这一操作会在正在被分片的图像上进行
                            cv2.rectangle(img_copy, (x, y), (x+patch_w, y+patch_h), (255, 255, 255), 2)
                        else:
                            cv2.imwrite(os.path.join(subfolder, f'x{x}-{x+patch_w}_y{y}-{y+patch_h}.jpg'), patch)
            if draw:
                # 循环完毕后，绘制分割后的图像
                plt.figure(dpi=400)
                plt.imshow(img_copy)
                plt.axis('off');

In [19]:
extract_images_from_folder(PATH, PATH_image)

In [20]:
extract_images_from_folder(PATH, PATH_mask)